In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [25]:
status = pd.read_csv('../../data/TCGA_metadata/TCGA_OV_HRDstatus.txt', sep='\t')
status.head()

,PatientID,Signature.1,Signature.2,Signature.3,Signature.5,Signature.8,Signature.13,Microhomology2,Microhomology2ratio,Del/ins-ratio,...,SBS39,SBS40,SBS41,ID1,ID2,ID4,ID8,HRDetect,BRCA-status_detailed,BRCA-status
0,TCGA-04-1331,30.1,0.0,185.5,85.5,21.0,0.0,9,0.428571,1.909091,...,0.0,0.000,48.603,0.000,0.0,0.0,32.000,0.974783,BRCA2_somatic_mutated+LOH,BRCA2_deficient
1,TCGA-04-1332,21.9,10.4,40.4,87.3,0.0,0.0,1,0.090909,2.285714,...,0.0,25.438,38.938,0.000,0.0,0.0,6.000,0.161715,intact,intact
2,TCGA-04-1336,13.6,0.0,80.9,49.4,0.0,0.0,8,0.888889,2.615385,...,0.0,44.129,0.000,1.515,0.0,0.0,11.485,0.974439,BRCA2_germline_mutated+LOH,BRCA2_deficient
3,TCGA-04-1341,0.0,0.0,50.7,42.7,0.0,0.0,0,0.000000,1.200000,...,0.0,0.000,31.444,0.000,0.0,0.0,11.000,0.088818,BRCA1_germline_mutated(UNK)+LOH,intact
4,TCGA-04-1342,0.0,0.0,545.4,419.9,0.0,0.0,3,0.062500,1.066667,...,0.0,0.000,0.000,9.176,0.0,0.0,83.824,0.168023,BRCA2_somatic_mutated(UNK)+LOH,intact


### Get columns of interest

In [26]:
filtered_status = status[['PatientID', 'HRDetect', 'BRCA-status',"Signature.1","Signature.2","Signature.3","Signature.5","Signature.8","Signature.13","Microhomology2","Microhomology2ratio","Del/ins-ratio","Del10-ratio","HRD-LOH","Telomeric.AI","LST","DBS2","DBS4","DBS5","DBS6","DBS9","SBS1","SBS2","SBS3","SBS5","SBS8","SBS13","SBS18","SBS26","SBS35","SBS38","SBS39","SBS40","SBS41","ID1","ID2","ID4","ID8"]].copy()
filtered_status.head()

,PatientID,HRDetect,BRCA-status,Signature.1,Signature.2,Signature.3,Signature.5,Signature.8,Signature.13,Microhomology2,...,SBS26,SBS35,SBS38,SBS39,SBS40,SBS41,ID1,ID2,ID4,ID8
0,TCGA-04-1331,0.974783,BRCA2_deficient,30.1,0.0,185.5,85.5,21.0,0.0,9,...,0.000,0.0,0.0,0.0,0.000,48.603,0.000,0.0,0.0,32.000
1,TCGA-04-1332,0.161715,intact,21.9,10.4,40.4,87.3,0.0,0.0,1,...,0.000,0.0,0.0,0.0,25.438,38.938,0.000,0.0,0.0,6.000
2,TCGA-04-1336,0.974439,BRCA2_deficient,13.6,0.0,80.9,49.4,0.0,0.0,8,...,16.588,0.0,0.0,0.0,44.129,0.000,1.515,0.0,0.0,11.485
3,TCGA-04-1341,0.088818,intact,0.0,0.0,50.7,42.7,0.0,0.0,0,...,25.414,0.0,0.0,0.0,0.000,31.444,0.000,0.0,0.0,11.000
4,TCGA-04-1342,0.168023,intact,0.0,0.0,545.4,419.9,0.0,0.0,3,...,0.000,0.0,0.0,0.0,0.000,0.000,9.176,0.0,0.0,83.824


In [27]:
Counter(filtered_status['BRCA-status'])

Counter({'BRCA2_deficient': 39,
         'intact': 268,
         'quiescent': 17,
         'BRCA1_deficient': 101})

In [28]:
len(np.unique(filtered_status['PatientID'])), len(filtered_status['PatientID']) 

(425, 425)

### Get slide paths

In [29]:
paths = list(Path('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/').glob('*.svs'))
paths[:5]

[PosixPath('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/TCGA-61-2008-02A-01-TS1.f04bb94a-6caf-4ba5-a918-08db1b26b964.svs'),
 PosixPath('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/TCGA-61-1724-01A-01-BS1.dc9700ec-ce3c-4286-80bd-a353c376977b.svs'),
 PosixPath('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/TCGA-23-1022-01A-02-BS2.d08fdaaf-e78f-4e6d-ba65-1663cc37ab02.svs'),
 PosixPath('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/TCGA-04-1361-01A-01-BS1.b4e0df05-3e78-4414-9430-b4d97a09befb.svs'),
 PosixPath('/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides/TCGA-25-1321-01A-01-TS1.3cdd3537-543e-41b9-b024-f4b843205c0e.svs')]

In [30]:
slide_info = np.array([['-'.join(path.name.split('-')[:3]), path.name.split('.')[0][-3:]] for path in paths])
slide_names, slide_types = slide_info[:, 0], slide_info[:, 1]

print(slide_types[:3])
print(slide_names[:3])

['TS1' 'BS1' 'BS2']
['TCGA-61-2008' 'TCGA-61-1724' 'TCGA-23-1022']


In [31]:
Counter(slide_types)

Counter({'TS1': 652, 'BS1': 663, 'BS2': 54, 'DX1': 106, 'DX2': 1, 'TSA': 5})

In [32]:
len(slide_names), len(np.unique(slide_names))

(1481, 590)

In [33]:
# NOTE: Check how many slides are in the status file
len(set(slide_names).intersection(set(filtered_status['PatientID']))) # There are many slides for each patient... Are we throwing these away?...

425

In [34]:
# NOTE: check how many slides has `DX` in their name
filtered_slide_names = [name for name, slide_type in zip(slide_names, slide_types) if 'DX' in slide_type]
len(filtered_slide_names)

107

In [35]:
# NOTE: check how many filtered slides are in the status file
len(set(filtered_slide_names).intersection(set(filtered_status['PatientID'])))

72

In [36]:
# NOTE: update the dataframe with the slide names corresponding to the status file
filtered_status = status[['PatientID', 'HRDetect', 'BRCA-status',"Signature.1","Signature.2","Signature.3","Signature.5","Signature.8","Signature.13","Microhomology2","Microhomology2ratio","Del/ins-ratio","Del10-ratio","HRD-LOH","Telomeric.AI","LST","DBS2","DBS4","DBS5","DBS6","DBS9","SBS1","SBS2","SBS3","SBS5","SBS8","SBS13","SBS18","SBS26","SBS35","SBS38","SBS39","SBS40","SBS41","ID1","ID2","ID4","ID8"]].copy()

filtered_status['slide_paths'] = [None] * len(filtered_status)
filtered_status['slide_types'] = [None] * len(filtered_status)

# I think here you are actually ignoring extra slides each aptient might have becuase you append to the patient id

for ind, row in filtered_status.iterrows():
    filtered_status.loc[ind, 'slide_paths'] = ','.join([str(path).split('/')[-1].replace('.svs', '') for path in paths if row['PatientID'] in str(path)])
    filtered_status.loc[ind, 'slide_types'] = ','.join([slide_type for name, slide_type in zip(slide_names, slide_types) if row['PatientID'] in name])

In [37]:
len(filtered_status) # this is 425 long but should be longer... only use one slide randomly for each patient...

425

In [38]:
# NOTE: do the same for the whole status file
updated_status = status.copy()

updated_status['slide_types'] = [None] * len(updated_status)
updated_status['slide_paths'] = [None] * len(updated_status)
updated_status['FFPE'] = [None] * len(updated_status)

for ind, row in updated_status.iterrows():
    updated_status.loc[ind, 'slide_types'] = ','.join([slide_type for name, slide_type in zip(slide_names, slide_types) if row['PatientID'] in name])
    updated_status.loc[ind, 'slide_paths'] = ','.join([str(path).split('/')[-1].replace('.svs', '')  for path in paths if row['PatientID'] in str(path)])

    updated_status.loc[ind, 'FFPE'] = True if 'DX1' in updated_status.loc[ind, 'slide_types'] or 'DX2' in updated_status.loc[ind, 'slide_types'] else False

In [39]:
updated_status[['PatientID', 'BRCA-status', 'slide_types', 'slide_paths', 'FFPE']].loc[updated_status['FFPE'] == True]

,PatientID,BRCA-status,slide_types,slide_paths,FFPE
142,TCGA-13-A5FU,intact,"DX1,TSA",TCGA-13-A5FU-01Z-00-DX1.9AD9E4B9-3F87-4879-BC0...,True
152,TCGA-23-1021,BRCA2_deficient,"BS1,TS1,DX1",TCGA-23-1021-01B-01-BS1.43618d42-5e31-4911-b67...,True
153,TCGA-23-1022,BRCA1_deficient,"BS2,TS1,BS1,DX1",TCGA-23-1022-01A-02-BS2.d08fdaaf-e78f-4e6d-ba6...,True
154,TCGA-23-1027,BRCA1_deficient,"DX1,BS2,BS1",TCGA-23-1027-01Z-00-DX1.53F9DFF4-6811-4184-B2F...,True
155,TCGA-23-1029,BRCA2_deficient,"BS1,DX1,TS1",TCGA-23-1029-01B-01-BS1.f0bd82cb-50ca-4cf3-bd2...,True
...,...,...,...,...,...
367,TCGA-5X-AA5U,intact,"TSA,DX1",TCGA-5X-AA5U-01A-01-TSA.9D54B87F-491D-4482-A0D...,True
421,TCGA-OY-A56P,intact,"TS1,DX1",TCGA-OY-A56P-01A-01-TS1.8A88DA85-D084-483A-955...,True
422,TCGA-OY-A56Q,intact,"DX1,TS1",TCGA-OY-A56Q-01Z-00-DX1.F1556F26-8845-4962-900...,True
423,TCGA-VG-A8LO,BRCA1_deficient,"DX2,DX1",TCGA-VG-A8LO-01A-02-DX2.9B58474C-DAC0-4D45-B13...,True


In [40]:
updated_status[['PatientID', 'BRCA-status', 'HRDetect', 'slide_types', 'slide_paths', 'FFPE']].loc[updated_status['FFPE'] == False]

,PatientID,BRCA-status,HRDetect,slide_types,slide_paths,FFPE
0,TCGA-04-1331,BRCA2_deficient,0.974783,"TS1,BS1",TCGA-04-1331-01A-01-TS1.76f57a19-5698-4204-bb1...,False
1,TCGA-04-1332,intact,0.161715,"BS1,TS1",TCGA-04-1332-01A-01-BS1.0f52bd8b-65e8-4648-b6b...,False
2,TCGA-04-1336,BRCA2_deficient,0.974439,"BS1,TS1",TCGA-04-1336-01A-01-BS1.d029552a-1480-4083-846...,False
3,TCGA-04-1341,intact,0.088818,"BS1,TS1",TCGA-04-1341-01A-01-BS1.85fe6a7e-fd57-4f26-a84...,False
4,TCGA-04-1342,intact,0.168023,"TS1,BS1",TCGA-04-1342-01A-01-TS1.66421418-fc94-4215-9ab...,False
...,...,...,...,...,...,...
416,TCGA-61-2111,intact,0.055567,"TS1,BS1,TS1,BS1",TCGA-61-2111-11A-01-TS1.9160fa41-6476-47cf-b93...,False
417,TCGA-61-2113,intact,0.009278,"TS1,BS1",TCGA-61-2113-01A-01-TS1.91aa5d6b-d95f-4465-b1a...,False
418,TCGA-61-2612,intact,0.763266,"BS1,TS1,BS1,TS1",TCGA-61-2612-11A-01-BS1.471ed8ea-a82f-4ee4-b6b...,False
419,TCGA-61-2613,intact,0.994690,"BS1,TS1,BS1,TS1",TCGA-61-2613-01A-01-BS1.a03e1b6b-eb1f-4956-8ed...,False


In [41]:
# NOTE:
# Dividie up samples like NERO paper...

dataset = []
for _, row in filtered_status.iterrows():
    for slide_id, slide_type in zip(row['slide_paths'].split(','), row['slide_types'].split(',')):
        if row['BRCA-status'] == 'BRCA1_deficient' or row['BRCA-status'] == 'BRCA2_deficient':
                brca_status = 'positive'
        elif row['BRCA-status'] == 'quiescent':
                    brca_status = 'negative'
        # elif row['BRCA-status'] == 'intact':
        #             brca_status = 'negative'
        sample = {
            'case_id': row['PatientID'],
            'slide_id': slide_id,
            'slide_type': slide_type,
            'brca_status': brca_status,
            'hrd_score': row['HRDetect'],
            'Signature.1' : row['Signature.1'],
            'Signature.2' : row['Signature.2'],
            'Signature.3' : row['Signature.3'],
            'Signature.5' : row['Signature.5'],
            'Signature.8' : row['Signature.8'],
            'Signature.13' : row['Signature.13'],
            'Microhomology2' : row['Microhomology2'],
            'Microhomology2ratio' : row['Microhomology2ratio'],
            'Del/ins-ratio' : row['Del/ins-ratio'],
            'Del10-ratio' : row['Del10-ratio'],
            'HRD-LOH' : row['HRD-LOH'],
            'Telomeric.AI' : row['Telomeric.AI'],
            'LST' : row['LST'],
         'DBS2' : row['DBS2'],
           'DBS4' : row['DBS4'], 'DBS5' : row['DBS5'],
             'DBS6' : row['DBS6'], 'DBS9' : row['DBS9'], 
             'SBS1' : row['SBS1'], 'SBS2' : row['SBS2'],
               'SBS3' : row['SBS3'], 'SBS5' : row['SBS5'], 
               'SBS8' : row['SBS8'], 'SBS13' : row['SBS13'], 
               'SBS18' : row['SBS18'], 'SBS26' : row['SBS26'],
                 'SBS35' : row['SBS35'], 'SBS38' : row['SBS38'],
                   'SBS39' : row['SBS39'], 'SBS40' : row['SBS40'], 
                   'SBS41' : row['SBS41'], 'ID1' : row['ID1'],
                     'ID2' : row['ID2'], 'ID4' : row['ID4'], 'ID8' : row['ID8']
        }

        dataset.append(sample)


dataset = pd.DataFrame(dataset)
scaler = StandardScaler()
genomic_features = ["Signature.1","Signature.2","Signature.3","Signature.5","Signature.8","Signature.13","Microhomology2","Microhomology2ratio","Del/ins-ratio","Del10-ratio","HRD-LOH","Telomeric.AI","LST","DBS2","DBS4","DBS5","DBS6","DBS9","SBS1","SBS2","SBS3","SBS5","SBS8","SBS13","SBS18","SBS26","SBS35","SBS38","SBS39","SBS40","SBS41","ID1","ID2","ID4","ID8"]
# log transform.  
# Log Transformation (adding a small value to avoid log(0))
dataset[genomic_features] = dataset[genomic_features].applymap(lambda x: np.log(x + 1))
# Standard Scaling
genomic_features_scaled = scaler.fit_transform(dataset[genomic_features])
dataset[genomic_features] = genomic_features_scaled
dataset = dataset.sample(frac=1, random_state=137).reset_index(drop=True)

dataset.head()

,case_id,slide_id,slide_type,brca_status,hrd_score,Signature.1,Signature.2,Signature.3,Signature.5,Signature.8,...,SBS26,SBS35,SBS38,SBS39,SBS40,SBS41,ID1,ID2,ID4,ID8
0,TCGA-36-2532,TCGA-36-2532-01A-01-BS1.acb0df6f-8ce4-4af7-a5d...,BS1,negative,0.005919,0.582374,2.433831,-2.793493,0.176646,-1.015064,...,-0.453437,-0.394365,-0.076842,-0.775756,-1.207158,-1.004812,-0.472679,-0.355317,-0.066808,-1.340148
1,TCGA-30-1856,TCGA-30-1856-01A-01-TS1.81cdd81e-5115-405e-875...,TS1,positive,0.142597,0.973324,-0.371777,-0.302210,0.112623,0.807007,...,-0.453437,-0.394365,-0.076842,-0.775756,0.886728,-1.004812,1.859399,-0.355317,-0.066808,-0.258520
2,TCGA-57-1994,TCGA-57-1994-01Z-00-DX1.A0798185-30C5-4C32-B1A...,DX1,positive,0.418119,-0.102017,-0.371777,-1.205535,-0.008223,1.038977,...,-0.453437,-0.394365,-0.076842,1.160019,-1.207158,0.786210,-0.472679,-0.355317,-0.066808,-0.924600
3,TCGA-29-1695,TCGA-29-1695-01A-01-BS1.06ff1b46-dadd-4aa2-a60...,BS1,positive,0.035185,0.854228,-0.371777,-0.404311,0.795527,-1.015064,...,1.918658,-0.394365,-0.076842,-0.775756,0.938052,0.652855,2.448534,-0.355317,-0.066808,-0.607790
4,TCGA-61-2092,TCGA-61-2092-01A-01-TS1.060203fd-43c4-4e4c-8b3...,TS1,positive,0.002039,0.767189,2.578503,0.232296,0.602694,-1.015064,...,-0.453437,-0.394365,-0.076842,-0.775756,-1.207158,0.679462,-0.472679,2.131420,-0.066808,0.026413


### BRCA1/2 vs qiesecnet+intact with only ffpe slides stratied by patients.  

In [42]:
#binary split no intact slides
FFPE_only = dataset[dataset['slide_type'].isin(["DX1","DX2","TS1","TS2"])]
FFPE_only = FFPE_only[FFPE_only["brca_status"] != 'intact'] #remove intact
FFPE_only["brca_status"].value_counts()

FFPE_only.to_csv('/mnt/ncshare/ozkilim/BRCA/data/tasks/BRCA_vs_quiescent_DX_TS.csv', index=False)
# Pick only FFPE slides with no intact.

In [43]:
# threshold = 0.7
# # Create a new column based on binary thresholding
# df['hrd_status'] = df['hrd_score'].apply(lambda x: 1 if x >= threshold else 0)
# df_hrd_status = df[["case_id","slide_id","hrd_status"]]
# df_hrd_status.to_csv("../data/raw_subsets/HRD_binary.csv",index=False)

### Generate more subsets

In [44]:
# #binary split no intact slides
# pos_neg = dataset[dataset["brca_status"] != "intact"]
# pos_neg.to_csv('../data/brca_dataset_pos_neg.csv', index=False)

# FFPE only
# FFPE_only = dataset[dataset['slide_type'].isin(["DX1","DX2"])]
# FFPE_only.to_csv('../data/brca_dataset_FFPE_NERO.csv', index=False)

# # TS only 
# TS_only = dataset[dataset['slide_type'].isin(["TS1","TS2"])] #
# TS_only.to_csv('../data/brca_dataset_TS_1_2.csv', index=False)


# # binary split and FFPE..

# FFPE_only_binary = pos_neg[pos_neg['slide_type'].isin(["DX1","DX2"])]
# FFPE_only_binary.to_csv('../data/brca_dataset_FFPE_binary.csv', index=False)


# TODO: 



In [45]:
26/(26+47) # braca similar to nero...% 

0.3561643835616438

In [46]:
# understand the HRD score vs the classes? vs preds?